In [689]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [690]:
import cufflinks as cf  # noqa
import numpy as np
import pandas as pd
import plotly.offline  # noqa
import plotly.express as px
from src.report import conf_int, t_test, f_test, general, general_assessment, other
from src.stats import LinearRegression, add_constant

In [691]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True, theme='white')

# Считывание данных

In [692]:
df = pd.read_excel('data/table.xlsx', usecols=[1, 2], names=['y', 'x'])
df

,y,x
0,310,390
1,350,385
2,330,545
3,425,680
4,502,810
5,360,780
6,420,790
7,505,785
8,280,400
9,305,530


## Парная корреляция

In [693]:
df.corr()

,y,x
y,1.00000,0.82022
x,0.82022,1.00000


# Линейная регрессия

## R

In [694]:
%Rpush Y X df

In [695]:
%%R

library(lmtest)

m = lm(Y ~ 1 + x, data=df)

sm = summary(m)

sm


Call:
lm(formula = Y ~ 1 + x, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.24065 -0.08328  0.03805  0.07103  0.15530 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 5.2892537  0.1035604  51.074  < 2e-16 ***
x           0.0010737  0.0001542   6.963 5.45e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1003 on 22 degrees of freedom
Multiple R-squared:  0.6879,	Adjusted R-squared:  0.6737 
F-statistic: 48.49 on 1 and 22 DF,  p-value: 5.454e-07



## Python

In [696]:
Y = df['y']
X = add_constant(df['x'])

In [697]:
model = LinearRegression(Y, X)

In [698]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     45.23
Date:                Sat, 09 Oct 2021   Prob (F-statistic):           9.27e-07
Time:                        06:59:16   Log-Likelihood:                -121.84
No. Observations:                  24   AIC:                             247.7
Df Residuals:                      22   BIC:                             250.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        132.1547     41.799      3.162      0.0

## Модель

In [699]:
general(model)

$ y_i = a + b_{1} x_{i1} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} $<br>
<br>
$ \hat{y_i} = 132.155 + 0.419 x_{i1} $<br>
<br>
$ S_\hat{a} = 41.799 $<br>
$ S_{\hat{b_{1}}} = 0.062 $

## Общая оценка

In [700]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 40.499 $<br>
$ 5 \% < 9.93_{S_{ei}} \% < 10 \% \rightarrow $ оценка: <b>7.5/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.673 $<br>
$ 0.5 < 0.673 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 8.396 $<br>
$ 5 \% < 8.396 < 10 \% \rightarrow $ оценка: <b>6/10</b>.

## Оценка модели на значимость

In [701]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 45.229 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.028 $
<br><br>
$ 9.27e-07 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [702]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.074 $
<br>

<br>
$ t_{a} = 3.162 $, $ p\text{-}value_{t_{a}} = 0.005 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = 6.725 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

## Доверительные интервалы

In [703]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (45.470; 218.840) $<br>
$ b_{1} : (0.289; 0.548) $

## Дополнительные оценки

In [704]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \tilde{b_j} = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.000 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \tilde{b_{1}} = 0.510 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.676 $

## График

In [705]:
linear_predicts = model.results.predict()

In [706]:
sorted_x = np.sort(df['x'])
sorted_y = np.sort(df['y'])
sorted_linear_predicts = np.sort(linear_predicts)

In [707]:
scatter_plot = px.scatter(x=df['x'], y=df['y'])
scatter_plot.add_scatter(x=sorted_x, y=sorted_linear_predicts)

## Интерпретация параметров

Линейная зависимость **потребительские расходы**:
- коэффициент 0.4186 означает, что каждая единица **средней заработной платы**
  увеличивает **потребительские расходы** на 418.6 рублей.
- при отсутствии **заработной платы и социальных выплат**
  **потребительские расходы** сохранятся на уровне 132 тыс. руб. (МРОТ).

# Гиперболическая регрессия

## R

In [708]:
%Rpush df

In [709]:
%%R

library(lmtest)

df['x'] = 1 / df['x']

m = lm(y ~ 1 + x, data=df)

sm = summary(m)

sm


Call:
lm(formula = y ~ 1 + x, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-88.059 -32.283   1.407  34.688  68.901 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)     610.79      37.04   16.49 7.22e-14 ***
x           -126931.52   22424.53   -5.66 1.08e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 45.17 on 22 degrees of freedom
Multiple R-squared:  0.5929,	Adjusted R-squared:  0.5744 
F-statistic: 32.04 on 1 and 22 DF,  p-value: 1.082e-05



## Python

In [710]:
Y = df['y']
X = add_constant(1 / df['x'])

In [711]:
model = LinearRegression(Y, X)

In [712]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.593
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     32.04
Date:                Sat, 09 Oct 2021   Prob (F-statistic):           1.08e-05
Time:                        06:59:18   Log-Likelihood:                -124.46
No. Observations:                  24   AIC:                             252.9
Df Residuals:                      22   BIC:                             255.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        610.7915     37.044     16.488      0.0

## Модель

In [713]:
general(model)

$ y_i = a + b_{1} x_{i1} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} $<br>
<br>
$ \hat{y_i} = 610.791 + -126931.519 x_{i1} $<br>
<br>
$ S_\hat{a} = 37.044 $<br>
$ S_{\hat{b_{1}}} = 22424.531 $

## Общая оценка

In [714]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 45.172 $<br>
$ 10 \% < 11.08_{S_{ei}} \% < 15 \% \rightarrow $ оценка: <b>3.5/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.593 $<br>
$ 0.5 < 0.593 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 9.196 $<br>
$ 5 \% < 9.196 < 10 \% \rightarrow $ оценка: <b>6/10</b>.

## Оценка модели на значимость

In [715]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 32.040 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.028 $
<br><br>
$ 1.08e-05 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [716]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.074 $
<br>

<br>
$ t_{a} = 16.488 $, $ p\text{-}value_{t_{a}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = -5.660 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

## Доверительные интервалы

In [717]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (533.967; 687.616) $<br>
$ b_{1} : (-173437.151; -80425.887) $

## Дополнительные оценки

In [718]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \tilde{b_j} = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \tilde{b_{1}} = -41108731.322 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \tilde{b_{1}} = 164846.965 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ \tilde{b_{1}} = -0.498 $

## График

In [719]:
hyperbolic_predicts = model.results.predict()

In [720]:
sorted_x = np.sort(df['x'])
sorted_y = np.sort(df['y'])
sorted_hyperbolic_predicts = np.sort(hyperbolic_predicts)

In [721]:
scatter_plot = px.scatter(x=df['x'], y=df['y'])
scatter_plot.add_scatter(x=sorted_x, y=sorted_hyperbolic_predicts)

# Степенная регрессия

## R

In [722]:
%Rpush df

In [723]:
%%R

library(lmtest)

df['y'] = log(df['y'])
df['x'] = log(df['x'])

m = lm(y ~ 1 + x, data=df)

sm = summary(m)

sm


Call:
lm(formula = y ~ 1 + x, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.22890 -0.08572  0.02361  0.08121  0.17685 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.02237    0.60443   3.346  0.00293 ** 
x            0.61457    0.09342   6.578 1.29e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1043 on 22 degrees of freedom
Multiple R-squared:  0.663,	Adjusted R-squared:  0.6477 
F-statistic: 43.28 on 1 and 22 DF,  p-value: 1.29e-06



## Python

In [724]:
Y = np.log(df['y'])
X = add_constant(np.log(df['x']))

In [725]:
model = LinearRegression(Y, X)

In [726]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     43.28
Date:                Sat, 09 Oct 2021   Prob (F-statistic):           1.29e-06
Time:                        06:59:21   Log-Likelihood:                 21.249
No. Observations:                  24   AIC:                            -38.50
Df Residuals:                      22   BIC:                            -36.14
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0224      0.604      3.346      0.0

## Модель

In [727]:
general(model)

$ y_i = a + b_{1} x_{i1} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} $<br>
<br>
$ \hat{y_i} = 2.022 + 0.615 x_{i1} $<br>
<br>
$ S_\hat{a} = 0.604 $<br>
$ S_{\hat{b_{1}}} = 0.093 $

## Общая оценка

In [728]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 0.104 $<br>
$ 0 \% < 1.74_{S_{ei}} \% < 2 \% \rightarrow $ оценка: <b>10/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.663 $<br>
$ 0.5 < 0.663 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 1.411 $<br>
$ 0 \% < 1.411 < 2 \% \rightarrow $ оценка: <b>10/10</b>.

## Оценка модели на значимость

In [729]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 43.276 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.028 $
<br><br>
$ 1.29e-06 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [730]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.074 $
<br>

<br>
$ t_{a} = 3.346 $, $ p\text{-}value_{t_{a}} = 0.003 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = 6.578 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

## Доверительные интервалы

In [731]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (0.769; 3.276) $<br>
$ b_{1} : (0.421; 0.808) $

## Дополнительные оценки

In [732]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \tilde{b_j} = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.327 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \tilde{b_{1}} = 0.755 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.663 $

## График

In [733]:
power_predicts = np.exp(model.results.predict())

In [734]:
sorted_x = np.sort(df['x'])
sorted_y = np.sort(df['y'])
sorted_power_predicts = np.sort(power_predicts)

In [735]:
scatter_plot = px.scatter(x=df['x'], y=df['y'])
scatter_plot.add_scatter(x=sorted_x, y=sorted_power_predicts)

# Показательная регрессия

## R

In [736]:
%Rpush df

In [737]:
%%R

library(lmtest)

df['y'] = log(df['y'])

m = lm(y ~ 1 + x, data=df)

sm = summary(m)

sm


Call:
lm(formula = y ~ 1 + x, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.24065 -0.08328  0.03805  0.07103  0.15530 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 5.2892537  0.1035604  51.074  < 2e-16 ***
x           0.0010737  0.0001542   6.963 5.45e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.1003 on 22 degrees of freedom
Multiple R-squared:  0.6879,	Adjusted R-squared:  0.6737 
F-statistic: 48.49 on 1 and 22 DF,  p-value: 5.454e-07



## Python

In [738]:
Y = np.log(df['y'])
X = add_constant(df['x'])

In [739]:
model = LinearRegression(Y, X)

In [740]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     48.49
Date:                Sat, 09 Oct 2021   Prob (F-statistic):           5.45e-07
Time:                        06:59:23   Log-Likelihood:                 22.170
No. Observations:                  24   AIC:                            -40.34
Df Residuals:                      22   BIC:                            -37.98
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.2893      0.104     51.074      0.0

## Модель

In [741]:
general(model)

$ y_i = a + b_{1} x_{i1} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} $<br>
<br>
$ \hat{y_i} = 5.289 + 0.001 x_{i1} $<br>
<br>
$ S_\hat{a} = 0.104 $<br>
$ S_{\hat{b_{1}}} = 0.000 $

## Общая оценка

In [742]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 0.100 $<br>
$ 0 \% < 1.67_{S_{ei}} \% < 2 \% \rightarrow $ оценка: <b>10/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.688 $<br>
$ 0.5 < 0.688 < 0.7 \rightarrow $ оценка: <b>3/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 1.373 $<br>
$ 0 \% < 1.373 < 2 \% \rightarrow $ оценка: <b>10/10</b>.

## Оценка модели на значимость

In [743]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 48.486 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.028 $
<br><br>
$ 5.45e-07 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - модель в целом <b>значима</b>.

## Оценка значимости параметров модели

In [744]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.074 $
<br>

<br>
$ t_{a} = 51.074 $, $ p\text{-}value_{t_{a}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.<br>$ t_{b_{1}} = 6.963 $, $ p\text{-}value_{t_{b_{1}}} = 0.000 $, $ p\text{-}value < \alpha \rightarrow $гипотеза $ H_0 $ <b>отвергается</b> - параметр <b>значим</b>.

## Доверительные интервалы

In [745]:
conf_int(model)

$ \hat{b_i} - \hat{\sigma} \cdot t_{табл} \leq b_i \leq \hat{b_i} + \hat{\sigma} \cdot t_{табл} $<br>
<br>
$ \alpha = 0.05 $<br>
<br>
$ a : (5.074; 5.504) $<br>
$ b_{1} : (0.001; 0.001) $

## Дополнительные оценки

In [746]:
other(model)

### Бета-коэффициенты.<br>
<br>
$ \tilde{b_j} = \hat{b_j} \cdot \frac{S_{x_{ij}}}{S_{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.000 $<br>
<br>
### Дельта-коэффициенты.<br>
<br>
$ \Delta_j = r_{y_i x_{ij}} \cdot \frac{\hat{b_j}}{R^2} $<br>
<br>
$ \tilde{b_{1}} = 0.001 $<br>
<br>
### Коэффициенты эластичности.<br>
<br>
$ Э_{j} = \hat{b_j} \cdot \frac{\bar{x_{ij}}}{\bar{y_i}} $<br>
<br>
$ \tilde{b_{1}} = 0.118 $

## График

In [747]:
exponential_predicts = np.exp(model.results.predict())

In [748]:
sorted_x = np.sort(df['x'])
sorted_y = np.sort(df['y'])
sorted_exponential_predicts = np.sort(exponential_predicts)

In [749]:
scatter_plot = px.scatter(x=df['x'], y=df['y'])
scatter_plot.add_scatter(x=sorted_x, y=sorted_exponential_predicts)

# Итоги

In [750]:
scatter_plot = px.scatter(x=df['x'], y=df['y'])
scatter_plot.add_scatter(x=sorted_x, y=sorted_linear_predicts, name='Линейная')
scatter_plot.add_scatter(x=sorted_x, y=sorted_hyperbolic_predicts, name='Гиперболическая')
scatter_plot.add_scatter(x=sorted_x, y=sorted_power_predicts, name='Степенная')
scatter_plot.add_scatter(x=sorted_x, y=sorted_exponential_predicts, name='Показательная')

## Субъективные и объективные оценки

$ (S_e, R^2, A, model, (a, b)) $

## Линейная регрессия. Оценки.

(7.5, 3, 6, +, (+, +))

## Гиперболическая регрессия. Оценки.

(3.5, 3, 6, +, (+, +))

## Степенная регрессия. Оценки.

(10, 3, 10, +, (+, +))

## Показательная регрессия. Оценки.

(10, 3, 10, +, (+, +))

# Выводы

✔ **Степенная** и **Показательная**

➖ **Линейная**

❌ **Гиперболическая**
